In [38]:
""" Let's first code up k = 1"""
from random import uniform
from math import log
from tqdm import tqdm
#inter-arrival times I ~ Exp(λ)
#λ, k, p_i
λ = 2
μ = 3
def gen_arrival():
    u = uniform(0,1)
    return -(1/λ)*log(u)

def gen_jobsize():
    u = uniform(0,1)
    return -(1/μ)*log(u)
def run():
    pis = [0] * 50
    t = 0
    N = 0
    t_complete = float('inf')
    t_arrive = gen_arrival()
    arrivals = 0
    t_100k = 0
    to_depart = 1
    while True:
        #job completes next
        if t_complete < t_arrive:
            t += t_complete
            N -= 1
            if t_100k:
                to_depart -= 1
            if t_100k and not to_depart:
                # print(t,max(pis))
                return t-t_100k,N,[pi / (t) for pi in pis]
            t_arrive -= t_complete
            pis[N] += t_complete
            t_complete = gen_jobsize() if N > 0 else float('inf')
        #job arrives next
        else:
            t += t_arrive
            N += 1
            pis[N] += t_arrive
            t_complete -= t_arrive
            if N==1 and t_complete == float('inf'):
                t_complete = gen_jobsize()
            t_arrive = gen_arrival()
            arrivals += 1
            if arrivals == 1_000_000:
                t_100k = t
                to_depart = N
ts,Ns = [],[]
n = 50
pi_true = []
for _ in tqdm(range(n)):
    T,Nf,pis = run()
    # print(f'T = {T}')
    ts.append(T)
    Ns.append(Nf)
    pi_true.append(pis)
    # print(pis)
avg_T, avg_N = sum(ts)/n, sum(Ns)/n
print(f'Average T of 100kth job: {avg_T}')
print(f'Average N at 100kth job: {avg_N}')
print(f'λT: {λ * avg_T}')
import numpy as np
pis = np.array(pi_true)
final = np.mean(pis,axis=0)
print(final)
for i in range(len(final) - 1):
    print(final[i+1]/final[i])
a = [final[i+1] / final[i] for i in range(len(final) - 1)]
a = [x for x in a if x != 'nan']
print(f'Average Ratio:{np.mean(a)}')


100%|██████████| 50/50 [00:38<00:00,  1.31it/s]

Average T of 100kth job: 1.1165228058304637
Average N at 100kth job: 2.38
λT: 2.2330456116609274
[1.33315756e-01 4.22435131e-01 1.48135941e-01 9.87312596e-02
 6.57883329e-02 4.38815524e-02 2.92516722e-02 1.94831042e-02
 1.30013945e-02 8.68281182e-03 5.76273341e-03 3.85135001e-03
 2.56366879e-03 1.70325123e-03 1.12655495e-03 7.49728237e-04
 5.03781604e-04 3.38596096e-04 2.28477115e-04 1.53637321e-04
 1.04243341e-04 7.07477203e-05 4.45034741e-05 3.08507092e-05
 1.99424200e-05 1.30647923e-05 8.95916328e-06 5.89981163e-06
 3.77938203e-06 2.37587828e-06 1.63204874e-06 1.47054672e-06
 8.61566540e-07 5.91601947e-07 5.25219925e-07 2.82612651e-07
 2.20283718e-07 3.67213973e-07 1.71405303e-07 1.48040480e-07
 1.00418670e-07 8.30257355e-08 6.16216875e-08 2.06534245e-08
 7.98248303e-09 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
3.168681208109207
0.3506714518480538
0.6664909219618277
0.6663374204581609
0.6670111620886991
0.666605227384885
0.6660509557372682
0.667316


/var/folders/cf/xxr0550s20j371tqcflz3qfc0000gn/T/ipykernel_69939/437588349.py:70: RuntimeWarning: invalid value encountered in scalar divide
  print(final[i+1]/final[i])
/var/folders/cf/xxr0550s20j371tqcflz3qfc0000gn/T/ipykernel_69939/437588349.py:71: RuntimeWarning: invalid value encountered in scalar divide
  a = [final[i+1] / final[i] for i in range(len(final) - 1)]


In [39]:
a = [x for x in a if x > 0]
np.mean(a)

0.7385005854217649

In [9]:
import numpy as np
x = np.array([[1,2,3],[4,5,6]])
np.mean(x,axis=0)

array([2.5, 3.5, 4.5])

##### Dropping System, $k$ servers, $k$ types of jobs
- degenerate case: $k = 1$, $\mu_1 = \mu$
    - dropping system
    - we know from Problem 17.5 that $\pi_n = \frac{\rho}{n} \cdot C$
        - can use as a sanity check